In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [287]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(a) for a in d.diagonal()]) * L

In [241]:
#find the \sqrt{n} by extending to degree 2 field L = K[x]/(x^2-n)
def square_root_ext(F,n):
    R = PolynomialRing(F,'x')
    f = R.0**2-n; assert f in R
    L.<a> = f.splitting_field()
    return sqrt(L(n))

In [267]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition)
        K.<a> = NumberField(list(set([a.minpoly() for a in Q.list()])))
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = square_root_ext(K,unitary_factor)
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [164]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [146]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    unitary_dft_matrix = matrix([flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]).transpose()
    if unitary_dft_matrix.base_ring() == SR:
        return unitary_dft_matrix.simplify_full()
    else:
        return unitary_dft_matrix

In [55]:
#QUESTION: is the SGA DFT unitary?

In [170]:
SGA = SymmetricGroupAlgebra(QQ,3)

In [171]:
G = SGA.group()

In [172]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [173]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [174]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [205]:
specht_module = SGA.specht_module(partition)

In [206]:
rho = specht_module.representation_matrix

In [232]:
group_size = SGA.group().cardinality()

In [284]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Rational Field

In [288]:
P.list()

[4/3, 2/3, 2/3, 4/3]

In [286]:
Q = unitary_change_of_basis(SGA,partition); Q

[1/2*sqrt(2) + 1/2*sqrt(2/3) 1/2*sqrt(2) - 1/2*sqrt(2/3)]
[1/2*sqrt(2) - 1/2*sqrt(2/3) 1/2*sqrt(2) + 1/2*sqrt(2/3)]

In [246]:
Q_list = Q.list();

In [275]:
Q_list_polys = list(set([a.minpoly() for a in Q.list()])); Q_list_polys

[x^4 - 4/3*x^2 + 1/9]

In [282]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/3

In [283]:
matrix(K,Q)

ValueError: need a real or complex embedding to convert a non rational element of a number field into an algebraic number

In [273]:
square_root_ext(K,unitary_factor)

1/3*a^2 - 2/3

In [279]:
hat(delta(G[3]),partition,SGA,unitary=True)

TypeError: unsupported operand parent(s) for *: 'Number Field in a with defining polynomial x^4 - 4*x^2 + 1' and 'Full MatrixSpace of 2 by 2 dense matrices over Symbolic Ring'

In [137]:
G[3]

[2, 3, 1]

In [136]:
rho(G[3])

[6 1]
[6 0]

In [134]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[6 1]
[6 0]

In [139]:
U_rho*conjugate_transpose_pos_char(U_rho)

7


[0 6]
[1 6]

In [208]:
unitary_dft(SymmetricGroupAlgebra(QQ,3)).base_ring()

AttributeError: 'sage.rings.number_field.number_field_element_quadratic.NumberFieldElement_quadratic_sqrt' object has no attribute 'square_root'

In [187]:
U_dft = unitary_dft(SGA); U_dft*U_dft.conjugate().transpose()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [188]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [92]:
#what are the eigenvalues?

In [160]:
U_dft = unitary_dft(SGA)
k.<a> = U_dft.charpoly().splitting_field()
eigs = matrix(k,U_dft).eigenvalues(extend=False); eigs

[4*a^5 + 5*a^4 + a^3 + 5*a^2 + 3*a, a^5 + 5*a^4 + 3*a^2 + a, 3*a^5 + 6*a^4 + 4*a^3 + 6*a + 5, 5*a^5 + 6*a^4 + 3*a^3 + 3*a^2 + 3*a + 5, 4*a^5 + 6*a^4 + 2*a^3 + 3*a^2 + 2*a + 4, 3*a^5 + 6*a^4 + 4*a^2 + 4*a + 1]

In [ ]:
eigs

In [93]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))